In [1]:
import pandas as pd
import numpy as np
import csv
import sys
sys.path.append("C:\\Users\\tiago\\OneDrive - Universidade de Aveiro\\3- Data Collection1")
import pickle
import neurokit2 as nk

# Acessing files functions
from reading_function import *
from stationarity_function import *
from neurokit_function import *

## Dividing Neutral signals in excerpts (2min each)

In [2]:
dict_data_S1 = pickle.load(open('dict_data_S1_file', 'rb'))
dict_emotion_S1 = pickle.load(open('dict_emotion_S1_file', 'rb'))
sampling_rate = 1000
dict_data_S2 = pickle.load(open('dict_data_S2_file', 'rb'))
dict_emotion_S2 = pickle.load(open('dict_emotion_S2_file', 'rb'))
sampling_rate = 1000

In [3]:
dict_split_S1 = dict_emotion_S1
dict_split_S2 = dict_emotion_S2

In [4]:
def spliting_signals (signal, sampling_rate):
    
    size = len(signal)
    
    splits = {}
    
    increment = round(size/5)
    
    for i in range (0, 5):
        st = i* increment
        en = (i+1)*increment
        small_signal = signal[st:en]
        splits[i+1]=small_signal
               
    return splits

In [5]:
def spliting_emotion (Emotion, dict_split, sampling_rate):
    
    for part in dict_split[Emotion][0]:
        signal = dict_split[Emotion][0][part]
        splits = spliting_signals (signal, sampling_rate)
        dict_split[Emotion][0][part] = splits
        
    return dict_split

In [6]:
dict_split_S1 = spliting_emotion ('Neutral', dict_split_S1, sampling_rate)
dict_split_S2 = spliting_emotion ('Neutral', dict_split_S2, sampling_rate)

In [7]:
pickle.dump( dict_split_S1, open( "dict_split_S1", "wb" ) )
pickle.dump( dict_split_S2, open( "dict_split_S2", "wb" ) )

In [8]:
dict_split_S1 = pickle.load( open( "dict_split_S1", "rb" ) )
sampling_rate=1000
dict_split_S2 = pickle.load( open( "dict_split_S2", "rb" ) )
sampling_rate=1000

#### Preprocesing signals

In [9]:
def all_preprocessed_features (dict_emotion, condition, sampling_rate):
    
    dic_features={}
        
    for part in dict_emotion[condition][0].keys():
        #print(part)
        dic_features[part]={}
        
        for split in dict_emotion[condition][0][part].keys():
            
            if (condition=='Fear') & (part=='C007') & (split==5):
                a = 118629
                b = len(dict_emotion[condition][0][part][split][:,1])
                
            elif (condition=='Fear') & (part=='C011') & (split==1):
                a=118534
                b = len(dict_emotion[condition][0][part][split][:,1])
            
            elif (condition=='Happy') & (part=='C008') & (split==4):
                a=121243
                b = len(dict_emotion[condition][0][part][split][:,1])
            
            elif (condition=='Happy') & (part=='C026') & (split==5):
                a=119540
                b = len(dict_emotion[condition][0][part][split][:,1])
                
            elif (condition=='Neutral') & (part=='C010') & (split==2):    
                a=len(dict_emotion[condition][0][part][split][:,0])
                b=131074
                
            elif (condition=='Neutral') & (part=='C014') & (split==5):
                a=132533
                b = len(dict_emotion[condition][0][part][split][:,1])
                
            else:
                a = len(dict_emotion[condition][0][part][split][:,0])
                b = len(dict_emotion[condition][0][part][split][:,1])
            
            #print(split)
            
            # Features
            MF_duration_activity, MF_peak_activity, MF_mean_activity, MF_area_activity, MF_amplitude_activity = signal_process (dict_emotion[condition][0][part][split][:,0], 'EMG', sampling_rate, a)
            TR_duration_activity, TR_peak_activity, TR_mean_activity, TR_area_activity, TR_amplitude_activity = signal_process (dict_emotion[condition][0][part][split][:,1], 'EMG', sampling_rate, b)
            eda_symp, scr_height, scr_amplitude, scr_risetime, scr_recoverytime, eda_tonic, eda_phasic = signal_process (dict_emotion[condition][0][part][split][:,2], 'EDA', sampling_rate, len(dict_emotion[condition][0][part][split][:,2]))
            ecg_Rpeaks, ecg_rate, t_duration = signal_process (dict_emotion[condition][0][part][split][:,3], 'ECG', sampling_rate, len(dict_emotion[condition][0][part][split][:,3]))
    
            dic_features[part][split]={'MF_duration_activity':MF_duration_activity,'MF_peak_activity':MF_peak_activity,
                                              'MF_mean_activity':MF_mean_activity,'MF_area_activity':MF_area_activity,
                                              'MF_amplitude_activity':MF_amplitude_activity,
                                              'TR_duration_activity':TR_duration_activity,'TR_peak_activity':TR_peak_activity,
                                              'TR_mean_activity':TR_mean_activity,'TR_area_activity':TR_area_activity,
                                              'TR_amplitude_activity':TR_amplitude_activity,
                                              'scr_height':scr_height,'scr_amplitude':scr_amplitude,'scr_risetime':scr_risetime,
                                              'scr_recoverytime':scr_recoverytime, 'eda_tonic':eda_tonic, 'eda_phasic':eda_phasic,
                                              'ecg_rate':ecg_rate,'t_duration':t_duration,
                                              'eda_symp':eda_symp, 'ecg_Rpeaks':ecg_Rpeaks}
      
    return dic_features

In [10]:
dict_split_neutral_S1 = all_preprocessed_features (dict_split_S1, 'Neutral', sampling_rate)
dict_split_neutral_S2 = all_preprocessed_features (dict_split_S2, 'Neutral', sampling_rate)

C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\tiago\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\User

In [15]:
# Creating files with Session 1 relevant information to keep:
filename_S1 = 'dict_split_neutral_S1'
file_S1 = open(filename_S1,'wb')
pickle.dump(dict_split_neutral_S1,file_S1)
file_S1.close()
 
# Creating files with Session 2 relevant information to keep:
filename_S2 = 'dict_split_neutral_S2'
file_S2 = open(filename_S2,'wb')
pickle.dump(dict_split_neutral_S2,file_S2)
file_S2.close()

#### Acessing preprocessed splited data

In [16]:
dict_split_neutral_S1 = pickle.load(open('dict_split_neutral_S1', 'rb'))
dict_split_neutral_S2 = pickle.load(open('dict_split_neutral_S2', 'rb'))

#### Feature Extraction

In [17]:
def all_features (dic_features, sampling_rate):
    lista=[]
        
    for participant in dic_features.keys():  
        
        for split in dic_features[participant].keys():
        
            # Features
            features_emg_mf = feature_extraction (dic_features[participant][split], 'EMG_MF', sampling_rate)
            features_emg_tr = feature_extraction (dic_features[participant][split], 'EMG_TR', sampling_rate)
            features_eda = feature_extraction (dic_features[participant][split], 'EDA', sampling_rate)
            features_ecg = feature_extraction (dic_features[participant][split], 'ECG', sampling_rate)
    
            part=pd.DataFrame({'ID participant':[participant], 'Excerpt':[split]})
            features_=pd.concat([part, features_emg_mf, features_emg_tr, features_eda, features_ecg],axis=1)
            lista.append(features_)    
      
    return lista

In [18]:
neutral_s1_features = all_features(dict_split_neutral_S1, sampling_rate)
neutral_s1_df = pd.DataFrame()
neutral_s1_features_df = neutral_s1_df.append(other=neutral_s1_features,ignore_index=True)

neutral_s2_features = all_features(dict_split_neutral_S2, sampling_rate)
neutral_s2_df = pd.DataFrame()
neutral_s2_features_df = neutral_s2_df.append(other=neutral_s2_features,ignore_index=True)

C:\Users\tiago\OneDrive - Universidade de Aveiro\3- Data Collection1\neurokit_function.py:206: RuntimeWarning: invalid value encountered in double_scalars
  skewness = (3*(mean-np.nanmedian(a)))/standard_deviation
C:\Users\tiago\OneDrive - Universidade de Aveiro\3- Data Collection1\neurokit_function.py:206: RuntimeWarning: invalid value encountered in double_scalars
  skewness = (3*(mean-np.nanmedian(a)))/standard_deviation
C:\Users\tiago\AppData\Local\Temp\ipykernel_18476\3385887210.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neutral_s1_features_df = neutral_s1_df.append(other=neutral_s1_features,ignore_index=True)
C:\Users\tiago\AppData\Local\Temp\ipykernel_18476\3385887210.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  neutral_s2_features_df = neutral_s2_df.append(other=neutral_s2_features,ignor

In [19]:
filepath_feat='neutral_features_excerpts_s1.xlsx'
writer=pd.ExcelWriter(filepath_feat, engine='xlsxwriter')
neutral_s1_features_df.to_excel(writer, sheet_name='neutral_s1', na_rep='nan')
writer.save()

filepath_feat='neutral_features_excerpts_s2.xlsx'
writer=pd.ExcelWriter(filepath_feat, engine='xlsxwriter')
neutral_s2_features_df.to_excel(writer, sheet_name='neutral_s2', na_rep='nan')
writer.save()